In [8]:
import  tushare as ts
# 获取代号为000300的股票价格
cons=ts.get_apis()
df1=ts.bar('000001', conn=cons, asset='INDEX', start_date='2018-01-01', end_date='')

2. 对于获取的数据按日期进行升序排列，因为我们要通过历史的情况预测未来的情况

In [19]:
df=df1.sort_index(ascending=True).astype(float)
df.head(10)

,code,open,close,high,low,vol,amount,p_change
datetime,,,,,,,,
2018-01-02,1.0,3314.03,3348.33,3349.05,3314.03,2022788.0,2.277885e+11,NaN
2018-01-03,1.0,3347.74,3369.11,3379.92,3345.29,2138361.0,2.583665e+11,0.62
2018-01-04,1.0,3371.00,3385.71,3392.83,3365.30,2069552.0,2.430908e+11,0.49
2018-01-05,1.0,3386.46,3391.75,3402.07,3380.25,2130606.0,2.481878e+11,0.18
2018-01-08,1.0,3391.55,3409.48,3412.73,3384.56,2361651.0,2.862132e+11,0.52
2018-01-09,1.0,3406.11,3413.90,3417.23,3403.59,1914885.0,2.382500e+11,0.13
2018-01-10,1.0,3414.11,3421.83,3430.21,3398.84,2090949.0,2.545154e+11,0.23
2018-01-11,1.0,3415.58,3425.35,3426.48,3405.64,1738121.0,2.184141e+11,0.10
2018-01-12,1.0,3423.88,3428.94,3435.42,3417.98,1740634.0,2.159615e+11,0.10


3.取开盘价，收盘价，最高价，最低价，交易量五个特征，并做标准化

df=df[["open","close","high","low","vol"]]
df=df.apply(lambda x:(x-min(x))/(max(x)-min(x)))

import torch
import torch.nn as nn
from torch.autograd import *
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

'''
def SeriesGen(N):
    x = torch.arange(1,N,0.01)
    return torch.sin(x)
'''
def SeriesGen():
    x = torch.FloatTensor(df['high'].to_numpy())
    return x
 
def trainDataGen(seq,k):
    dat = list()
    L = len(seq)
    for i in range(L-k-1):
        indat = seq[i:i+k]
        outdat = seq[i+1:i+k+1]
        dat.append((indat,outdat))
    return dat
 
def ToVariable(x):
    tmp = torch.FloatTensor(x)
    return Variable(tmp)
 
y = SeriesGen()
dat = trainDataGen(y.numpy(),10)
 
 
class LSTMpred(nn.Module):
 
    def __init__(self,input_size,hidden_dim):
        super(LSTMpred,self).__init__()
        self.input_dim = input_size
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_size,hidden_dim)
        self.hidden2out = nn.Linear(hidden_dim,1)
        self.hidden = self.init_hidden()
 
    def init_hidden(self):
        return (Variable(torch.zeros(1, 1, self.hidden_dim)),
                Variable(torch.zeros(1, 1, self.hidden_dim)))
 
    def forward(self,seq):
        lstm_out, self.hidden = self.lstm(
            seq.view(len(seq), 1, -1), self.hidden)
        outdat = self.hidden2out(lstm_out.view(len(seq),-1))
        return outdat
 
 
model = LSTMpred(1,6)
loss_function = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
 
for epoch in range(10):
    print(epoch)
    for seq, outs in dat[:700]:
        seq = ToVariable(seq)
        outs = ToVariable(outs)
        #outs = torch.from_numpy(np.array([outs]))
 
        optimizer.zero_grad()
 
        model.hidden = model.init_hidden()
 
        modout = model(seq)
 
        loss = loss_function(modout, outs)
        loss.backward()
        optimizer.step()
 
predDat = []
for seq, trueVal in dat[700:]:
    seq = ToVariable(seq)
    trueVal = ToVariable(trueVal)
    predDat.append(model(seq)[-1].data.numpy()[0])


y.shape,len(dat)

fig = plt.figure()
plt.plot(y.numpy())
plt.plot(list(range(len(predDat))), predDat) 
plt.show()



https://blog.csdn.net/hustchenze/article/details/78696771?ops_request_misc=&request_id=&biz_id=102&utm_term=pytorch%201.4%20lstm&0utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduweb~default-5-78696771

https://mp.weixin.qq.com/s?__biz=MzA5NDM4MjMxOQ==&mid=2447578969&idx=1&sn=1ae03db749b56b1d2a140e1369bd8dba&chksm=8458c6d9b32f4fcf2ade01c726fa734476a9e173eb1189cc3a4eef3e0d350a635ea986e8f35c&mpshare=1&scene=1&srcid=0212XZYGJ80skfO7hOjclOb7&pass_ticket=4Ny3sz5foHRLUkvLoDFapNEMyCtao9JWtiG8SlAqvSYQiUvIQsIROlNPJ5l3iFyN#rd
https://blog.csdn.net/yangwohenmai1/article/details/84874197?utm_medium=distribute.pc_relevant.none-task-blog-baidujs-2

import torch
import torch.nn as nn
from torch.autograd import *
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
 
def SeriesGen(N):
    x = torch.arange(1,N,0.01)
    return torch.sin(x)
 
def trainDataGen(seq,k):
    dat = list()
    L = len(seq)
    for i in range(L-k-1):
        indat = seq[i:i+k]
        outdat = seq[i+1:i+k+1]
        dat.append((indat,outdat))
    return dat
 
def ToVariable(x):
    tmp = torch.FloatTensor(x)
    return Variable(tmp)
 
y = SeriesGen(10)
dat = trainDataGen(y.numpy(),10)
 
 
class LSTMpred(nn.Module):
 
    def __init__(self,input_size,hidden_dim):
        super(LSTMpred,self).__init__()
        self.input_dim = input_size
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_size,hidden_dim)
        self.hidden2out = nn.Linear(hidden_dim,1)
        self.hidden = self.init_hidden()
 
    def init_hidden(self):
        return (Variable(torch.zeros(1, 1, self.hidden_dim)),
                Variable(torch.zeros(1, 1, self.hidden_dim)))
 
    def forward(self,seq):
        lstm_out, self.hidden = self.lstm(
            seq.view(len(seq), 1, -1), self.hidden)
        outdat = self.hidden2out(lstm_out.view(len(seq),-1))
        return outdat
 
 
model = LSTMpred(1,6)
loss_function = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
 
for epoch in range(10):#设置训练次数
    #print(epoch)
    for seq, outs in dat[:700]:
        seq = ToVariable(seq)
        outs = ToVariable(outs)
        #outs = torch.from_numpy(np.array([outs]))
 
        optimizer.zero_grad()
 
        model.hidden = model.init_hidden()
 
        modout = model(seq)
 
        loss = loss_function(modout, outs)
        loss.backward()
        optimizer.step()
 
predDat = []
for seq, trueVal in dat[700:]:
    seq = ToVariable(seq)
    trueVal = ToVariable(trueVal)
    predDat.append(model(seq)[-1].data.numpy()[0])
 
fig = plt.figure()
plt.plot(y.numpy())
plt.plot(list(range(700,889)),predDat)
plt.show()


In [10]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [11]:
class RegressionNeuralNetwork(torch.nn.Module): # torch.nn.Module net的主模块
    def __init__(self,n_features,n_hidden,n_output):
        '''
        n_features 数据个数
        n_hidden 隐藏层的神经元的个数
        n_output
        '''
        super(RegressionNeuralNetwork,self).__init__() #要继承模块的信息
        self.hidden = torch.nn.Linear(n_features,n_hidden) # 隐藏层，输入的内容是他有多少哥输入和输出
        self.predict = torch.nn.Linear(n_hidden,n_output) # 输出预测层，输入的内容是

    def forward(self,x):
        '''前向传播'''
        x = F.relu(self.hidden(x)) #过一遍hidden传入上面定义的线性神经网络处理的函数中
        x = self.predict(x)
        return x

In [37]:
tensor_data = torch.FloatTensor(df.to_numpy())[1:]
tensor_data_y = tensor_data[:,-1]
tensor_data_x = tensor_data[:,:-1][:,None]
n = 200
train_x , train_y , test_x , test_y = tensor_data_x[:n],tensor_data_y[:n],tensor_data_x[n:],tensor_data_y[n:]
nn_features = train_x.shape[-1]
nn_hidden = 10
nn_output = nn_features
train_x.shape , train_y.shape , test_x.shape , test_y.shape

(torch.Size([200, 1, 7]),
 torch.Size([200]),
 torch.Size([397, 1, 7]),
 torch.Size([397]))

In [38]:
net = RegressionNeuralNetwork(n_features=nn_features,n_hidden=nn_hidden,n_output=nn_output)
'''
optimizer 设置优化器
lr是学习率 这里设置为0.005
'''
optimizer = torch.optim.SGD(net.parameters(),lr=0.005)
'''
设置损失函数
MSELoss 平方误差和做回归的损失函数比较合适
'''
loss_func = torch.nn.MSELoss()
'''训练神经网络10000次'''
for t in range(10000):
    '''调用图神经网络预测，输入x'''
    prediction = net(test_x) 
    '''计算y值的误差,传入预测值和y的实际值'''
    loss = loss_func(prediction,y)
    '''梯度下降的终点设置为0'''
    optimizer.zero_grad() #
    '''设置反向传播'''
    loss.backward() #
    ''''''
    optimizer.step()

RuntimeError: The size of tensor a (7) must match the size of tensor b (900) at non-singleton dimension 2